In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime as dt

from glob import iglob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = r'P:\000868684 - Hearts and Parks\Pattern Health Download\Summer 2021 data download - post study closure\**\*.csv'

df = pd.concat((pd.read_csv(f) for f in iglob(path, recursive=True)), ignore_index=True)

# Get MMI data
df_mmi = df[df['Type'] == 'meanMotionIntensity']

df_mmi['Start Time'] = df_mmi['Start Time'].apply(pd.to_datetime)

df_mmi['End Time'] = df_mmi['End Time'].apply(pd.to_datetime)

df_mmi['Start_Time_Offset'] = df_mmi['Start Time'] + pd.to_timedelta(df_mmi['Time Zone Offset'], unit='s')
df_mmi['End_Time_Offset'] = df_mmi['End Time'] + pd.to_timedelta(df_mmi['Time Zone Offset'], unit='s')

# get steps data
df_steps = df[df['Type'] == 'steps']
df_steps['User UUID'].unique()

df_steps['Start Time'] = df_steps['Start Time'].apply(pd.to_datetime)

df_steps['End Time'] = df_steps['End Time'].apply(pd.to_datetime)

In [ ]:
df_steps['Start_Time_Offset'] = df_steps['Start Time'] + pd.to_timedelta(df_steps['Time Zone Offset'], unit='s')
df_steps['End_Time_Offset'] = df_steps['End Time'] + pd.to_timedelta(df_steps['Time Zone Offset'], unit='s')

# df['Start_Time_Offset'] = df.loc[:,['Start Time','Time Zone Offset']].sum(axis=1)

df_merge_steps_mi = pd.merge(df_steps, df_mmi, left_on=['User UUID', 'Start_Time_Offset', 'End_Time_Offset'], right_on=['User UUID', 'Start_Time_Offset', 'End_Time_Offset'])
df_merge_steps_mi['User UUID'].unique()

In [ ]:
len(df_merge_steps_mi)

df_merge_steps_mi

df_merge_steps_mi.columns

In [ ]:
# Remove value_y from duplicate subset
df_merge_dup = df_merge_steps_mi.drop_duplicates(subset=['User UUID', 'Start_Time_Offset', 'End_Time_Offset', 'Value_x'], keep='first')
df_merge_dup

sixtyone_epoch = df_merge_dup[df_merge_dup['User UUID']=='u-22Kw0lWnvI27MXMuRggpVA']
sixtyone_epoch = sixtyone_epoch[sixtyone_epoch['Start Time_x'] == '2021-02-28T15:45:00Z']
sixtyone_epoch[['User UUID', 'End Time_x', 'End Time_y', 'Value_y', 'Value_x']]

# sixtyone_epoch = df_merge_steps_mi[df_merge_steps_mi['User UUID'] == 'u-22Kw0lWnvI27MXMuRggpVA']
# sixtyone_epoch[sixtyone_epoch['Start Time'] == '2021-02-28T15:45:00Z'][['User UUID', 'Start Time', 'End Time', 'Value']]

# MMIT value is slightly different
# sixtyone_epoch[sixtyone_epoch['Start Time_x'] == '2021-02-28T15:45:00Z']['Value_y'].unique()

# df_merge_dup['User UUID'].unique()

In [4]:
df_merge_dup['End Time'] = df_merge_dup['End Time_x'].apply(pd.to_datetime)

df_merge_dup['Start_Time_Date_Offset_x'] = df_merge_dup['Start Time Offset'].dt.date
df_merge_dup['End_Time_Date_Offset_x'] = df_merge_dup['End Time Offset'].dt.date

df_merge_dup['month_year'] = df_merge_dup['Start Time Offset'].dt.to_period('M')

# df_merge_dup['Total_Steps_Per_Day'] = df_merge_dup.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform(sum())

df_merge_dup['Total_Steps_Per_Day_pre_missing'] = df_merge_dup.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform('sum')

# df_merge_steps_mi_dup_z_nz_2['Average_Monthly_Steps_pre_missing'] = df_merge_steps_mi_dup_z_nz_2.groupby([' User UUID', 'month_year'])['Total_Steps_Per_Day'].transform('mean')

df_merge_dup['Mon_Year'] = df_merge_dup['Start_Time_Offset'].dt.strftime('%b-%Y')

df_merge_dup['Month_Year_Name'] = df_merge_dup['Mon_Year']

df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] == dt.date(2020, 3, 15)), 'Month_Year_Name'] = '1-14 Mar-2020'
df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] > dt.date(2020, 3, 15)) & (df_merge_dup['Start_Time_Date_Offset_x'] <= dt.date(2020, 3, 31)), 'Month_Year_Name'] = '15-31 Mar-2020'
df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] == dt.date(2020, 4, 1)), 'Month_Year_Name'] = df_merge_dup.Month_Year_Name

df_merge_dup['Average_Monthly_Steps_Lockdown_pre_missing'] = df_merge_dup.groupby(['User UUID', 'Month_Year_Name'])['Total_Steps_Per_Day_pre_missing'].transform('mean')


In [ ]:
df_zero = df_merge_dup[df_merge_dup['Value_y'] < 1]
df_zero['Value_y'].unique()

df_non_zero = df_merge_dup[df_merge_dup['Value_y'] > 1]
df_non_zero
df_non_zero['User UUID'].unique()

# Calculate missingness
# df_steps_missing = df_merge_steps_mi_dup_non_zero
df_steps_missing = df_non_zero

df_steps_missing['Num_of_days'] = 0
df_steps_missing['total_percent_missing'] = 0
df_steps_missing['percent_missing_per_day'] = 0
df_steps_missing['Epochs_per_day'] = 0



In [ ]:


df_steps_missing['Valid Day'] = 'no'
df_steps_missing.loc[(df_steps_missing['Epochs_per_day'] > 40), 'Valid Day'] = 'yes'

# df_steps_missing_Lockdown_all_years_copy = df_steps_missing

# restrict time
df_all_periods = df_steps_missing[(df_steps_missing['Start_Time_Date_Offset_x'] > dt.date(2019, 3, 1)) & (df_steps_missing['Start_Time_Date_Offset_x'] < dt.date(2021, 6, 30))]

print(df_all_periods['Start_Time_Date_Offset_x'].min())
print(df_all_periods['Start_Time_Date_Offset_x'].max())

df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] < dt.date(2020, 3, 15)) & (df_all_periods['Start_Time_Date_Offset_x'] > dt.date(2019, 3, 1)), 'closure_status'] = 'Pre Closure'
df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] < dt.date(2021, 3, 31)) & (df_all_periods['Start_Time_Date_Offset_x'] > dt.date(2020, 3, 15)), 'closure_status'] = 'During Closure'
df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] > dt.date(2021, 3, 31)), 'closure_status'] = 'Post Closure'

print(df_all_periods.groupby('closure_status')['Start_Time_Date_Offset_x'].min())
print(df_all_periods.groupby('closure_status')['Start_Time_Date_Offset_x'].max())

In [6]:
df_valid_days = df_all_periods[df_all_periods['Valid Day'] == 'yes']

df_valid_days['Num_of_days_valid_epochs'] = df_valid_days.groupby(['User UUID'])['Start_Time_Date_Offset_x'].transform('nunique')

df_valid_days_daycount = df_valid_days[df_valid_days['Num_of_days_valid_epochs'] > 60]

df_valid_days_daycount['Epochs_per_day'].min()
df_valid_days_daycount['Epochs_per_day'].max()

epoch_count = df_all_periods
epoch_count['Num_of_days_valid_epochs'] = epoch_count.groupby(['User UUID'])['Start_Time_Date_Offset_x'].transform('nunique')

epoch_count = df_all_periods.drop_duplicates(subset=['User UUID', 'Num_of_days_valid_epochs'], keep='first')


print(range(len(epoch_count_only)))
print(194/2)

epoch_count_only.max()


In [ ]:
# plt.figure(figsize=(100,40))
plt.figure(figsize=(60,20))
sns.set(font_scale=5)
default_x_ticks = range(len(epoch_count_only))
fig, ax = plt.subplots(figsize=(60,20))
ax.hist(epoch_count_only['Num_of_days_valid_epochs'], bins=61, cumulative=-1, density=False)
ax.set_facecolor('white')

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 120]['User UUID'].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 61].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 40].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 70].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 80].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 90].nunique()

In [ ]:
# Heatmap
df_epoch_dup = df_all_periods.drop_duplicates(subset=['User UUID', 'Start_Time_Date_Offset_x'], keep='last', inplace=False)
df2 = df_epoch_dup.groupby(['User UUID', 'Epochs_per_day'])['Start_Time_Date_Offset_x'].count()

df3 = df2.reset_index()

df3['Epochs_per_day'] = df3['Epochs_per_day'].replace(np.nan, 0)
df3['Start_Time_Date_Offset_x'] = df3['Start_Time_Date_Offset_x'].replace(np.nan, 0)

df_pivot = df3.pivot_table(values='User UUID', index=['Epochs_per_day'], columns=['Start_Time_Date_Offset_x'], aggfunc=pd.Series.nunique)
df_pivot.replace(np.nan, 0, inplace=True)

sns.set(rc={"figure.figsize": (20, 7)})
sns.set_context("paper", rc={"font.size":90,"axes.titlesize":50,"axes.labelsize":20})
sns.set(rc={"axes.facecolor":"snow", "figure.facecolor":"white"})
ax = sns.heatmap(df_pivot, cmap="rocket_r", mask=(df_pivot==0))
ax.invert_yaxis()

df_valid_days_daycount['User UUID'].nunique()

df_valid_days_daycount['Total_Steps_Per_Day_post_missing'] = df_valid_days_daycount.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Total_Steps_Per_Day'].transform('sum')

df_valid_days_daycount['Average_Monthly_Steps_Lockdown_post_missing'] = df_valid_days_daycount.groupby(['User UUID', 'Month_Year_Name'])['Total_Steps_Per_Day_post_missing'].transform('mean')

# 

In [ ]:
df_valid_days_daycount_dup = df_valid_days_daycount.drop_duplicates(subset=['User UUID', 'Average_Monthly_Steps_Lockdown_post_missing', 'Month_Year_Name'], keep='first')
df_valid_days_daycount_dup

# Min
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].min()

# Max
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].max()

# Mean
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].mean()

# Std
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].std()

# Users
df_valid_days_daycount_dup.groupby('closure_status')['User UUID'].nunique()

In [ ]:
df_valid_days_daycount_dup_month = df_valid_days_daycount_dup
df_valid_days_daycount_dup_month['average_steps_per_month_post_missing'] = df_valid_days_daycount_dup_month.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')
df_valid_days_daycount_dup_month = df_valid_days_daycount_dup_month.drop_duplicates(subset=['average_steps_per_month_post_missing', 'Month_Year_Name'], keep='first')
df_valid_days_daycount_dup_month

df_valid_days_daycount_dup_month['Month_Year_Name'].unique()

# Min
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].min()

# Max
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].max()

# Mean
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].mean()

# Std
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].std()

# Users
df_valid_days_daycount_dup_month.groupby('closure_status')['User UUID'].nunique()

# Exclude summer for TOD comparison with overall
df_valid_days_daycount_dup_month_non_sum = df_valid_days_daycount_dup_month[(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jun-2020') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jul-2020') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Aug-2020') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jun-2019') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jul-2019') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Aug-2019') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jun-2021') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jul-2021') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Aug-2021')]
df_valid_days_daycount_dup_month_non_sum

# Mean
df_valid_days_daycount_dup_month_non_sum.groupby('closure_status')['average_steps_per_month_post_missing'].mean()

In [ ]:
# Figure
# df_steps_missing_Lockdown_all_years_valid_days_daycount_fig4['year_only'] = df_steps_missing_Lockdown_all_years_valid_days_daycount

df_valid_days_daycount_dup_fig = df_valid_days_daycount_dup
df_valid_days_daycount_dup_fig['pre_avg'] = 8809.83

plt.figure(figsize=(60, 20))
sns.set(font_scale=5)
sns.set_style("white")
sns.set_palette("Set1", 8, .75)
ax = sns.lineplot(x='Month_Year_Name', y='Average_Monthly_Steps_Lockdown_post_missing', 
                  data=df_valid_days_daycount_dup_fig, color='darkcyan', 
                  linewidth=15, sort=False)
sns.lineplot(x='Month_Year_Name', y='pre_avg', 
             data=df_valid_days_daycount_dup_fig, color='grey', 
             linestyle='--', linewidth=10, sort=False)

plt.setp(ax.get_xticklabels(), rotation=90)
ax.set_xlabel("Month", fontdict={'fontsize': 60, 'fontweight': 'bold'}, labelpad=-2)
ax.set_ylabel("Average Daily Number of Steps Per Month", fontdict={'fontsize': 55, 'fontweight': 'bold'})
ax.set_title("Average Daily Step Values for Participants (Non zero MMI, after missingness analysis)", fontdict={'fontsize': 60, 'fontweight': 'bold'})
ax.axvline("1-14 Mar-2020", color="firebrick", linestyle="--", lw=10)
ax.legend(loc="upper right", frameon=True, fontsize=50)
plt.setp(ax.get_legend().get_texts(), fontsize='50') # for Legend text
plt.setp(ax.get_legend().get_title(), fontsize='50') # for Legend title

In [ ]:
# Statistical test
steps_pre_closure = df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status'] == 'Pre Closure']

steps_pre_closure['Monthly_Avg'] = steps_pre_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_pre_closure['month_num'] = steps_pre_closure['Start_Time_Offset'].dt.month
# steps_pre['month_num'].unique()
steps_pre_closure = steps_pre_closure.sort_values(by=['month_num'])
# steps_pre_closure

steps_pre_closure_col = steps_pre_closure[['Month_Year_Name', 'Monthly_Avg']]
steps_pre_closure_dup = steps_pre_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
# steps_pre_closure_dup

steps_dur_closure = df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status'] == 'During Closure']

steps_dur_closure['Monthly_Avg'] = steps_dur_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_dur_closure['month_num'] = steps_dur_closure['Start_Time_Offset'].dt.month
# steps_pre['month_num'].unique()
steps_dur_closure = steps_dur_closure.sort_values(by=['month_num'])
# steps_dur_closure

steps_dur_closure_col = steps_dur_closure[['Month_Year_Name', 'Monthly_Avg']]
steps_dur_closure_dup = steps_dur_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
# steps_dur_closure_dup

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_dup['Monthly_Avg'], steps_pre_closure_dup['Monthly_Avg'])
print(U1)
print(p)
print(p * 100)

In [ ]:
# Time of day analysis
df_closure_tod = df_valid_days_daycount

df_closure_tod['Start Time Only Time'] = df_closure_tod['Start_Time_Offset'].dt.strftime("%H:%M:%S")
# all_data_merged_steps[all_data_merged_steps['Start Time Only Time'] > '16:15:00']
# base on data instead
df_closure_tod['Time_Of_Day'] = "Night"
df_closure_tod.loc[df_closure_tod['Start Time Only Time'] < '11:00:00', 'Time_Of_Day'] = 'Morning'
df_closure_tod.loc[(df_closure_tod['Start Time Only Time'] >= '07:00:00') &
                   (df_closure_tod['Start Time Only Time'] < '16:00:00'), 'Time_Of_Day'] = 'Afternoon'
df_closure_tod.loc[(df_closure_tod['Start Time Only Time'] >= '11:00:00') &
                   (df_closure_tod['Start Time Only Time'] < '20:00:00'), 'Time_Of_Day'] = 'Evening'

# aft = df_steps_missing_all_dates_copy_fig3[df_steps_missing_all_dates_copy_fig3['Time_Of_Day'] == 'Evening']
# aft['Start Time Only Time'].unique()

df_closure_tod['wkday_Num'] = df_closure_tod['Start_Time_Offset'].dt.weekday
df_closure_tod['wkday_Num'].unique()

df_closure_tod_wkday = df_closure_tod[df_closure_tod['wkday_Num'].isin([0,1,2,3,4])]
df_closure_tod_wkday['wkday_Num'].unique()

df_closure_tod_wkday[['wkday_Num', 'Start_Time_Offset']]

In [ ]:

df_closure_tod_wkday_school = df_closure_tod_wkday[df_closure_tod_wkday['Time_Of_Day'].isin(['Morning', 'Afternoon'])]
df_closure_tod_wkday_school_non_sum = df_closure_tod_wkday_school[~df_closure_tod_wkday_school['Mon_Year'].isin(['Jun-2020', 'Jul-2020', 'Aug-2020', 'Jun-2019', 'Jul-2019', 'Aug-2019', 'Jun-2021', 'Jul-2021', 'Aug-2021'])]
df_closure_tod_wkday_school_non_sum['Daily_Sum_TOD'] = df_closure_tod_wkday_school_non_sum.groupby(['Start_Time_Date_Offset_x', 'User_UUID'])['Steps'].transform('sum')
df_closure_tod_wkday_school_non_sum['mean_monthly_steps_school'] = df_closure_tod_wkday_school_non_sum.groupby(['User_UUID', 'Month_Year_Name'])['Daily_Sum_TOD'].transform('mean')

df_closure_tod_wkday_school_non_sum_dup = df_closure_tod_wkday_school_non_sum.drop_duplicates(subset=['User_UUID', 'mean_monthly_steps_school'])
df_closure_tod_wkday_school_non_sum_dup

In [ ]:
# Max
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].max()

# Mean
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].mean()

# Std
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].std()

# Users
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
df_closure_tod_fig = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] != 'Post Closure']
plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {
    "Pre Closure": "mediumpurple",
    "During Closure": "lightcoral"
#     "#2020": "salmon"
}

ax = sns.boxplot(x="Month_Year_Name", y="mean_monthly_steps_school", palette=palette, hue="closure_status",
                 data=df_closure_tod_fig, linewidth=14,
                 showfliers=False, dodge=False, width=0.5,
                 showmeans=True)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# ax.set_title('Average Step Values for Participants Before and After Lockdown For School Times')
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times',
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)

In [ ]:
df_closure_tod_fig = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] != 'Post Closure']
fig_2_df_closure_tod = df_closure_tod_fig
# Following lines are not completely visible in the image, but here's the visible part:
fig_2_df_closure_tod['Month Year Name_Closure'] = fig_2_df_closure_tod['Month Year Name'].str[0:3]
fig_2_df_closure_tod.loc[(fig_2_df_closure_tod['Month Year Name_Closure'] == 'Mar'), 'Month Year Name_Closure'] = 'Mar-2020'
fig_2_df_closure_tod.loc[(fig_2_df_closure_tod['Month Year Name_Closure'] == 'Mar'), 'Month Year Name_Closure'] = 'Mar-'
fig_2_df_closure_tod['month only'] = pd.Categorical(fig_2_df_closure_tod['Month Year Name_Closure'],
                                                    categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)

In [ ]:
# Commented out line
# df_closure_tod_fig=df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status']=='Post Closure']

plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {"Pre Closure": "mediumpurple",
           "During Closure": "lightcoral",
           "#2020": "salmon"}

ax = sns.boxplot(x="month_only", y="mean_monthly_steps_school", palette=palette, hue="closure_status",
                 data=fig_2_df_closure_tod, linewidth=14, showfliers=False, dodge=False, width=0.5, showmeans=True)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times', 
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)

In [ ]:
# New one value per month
df_closure_tod_wkday_school_non_sum_dup_month = df_closure_tod_wkday_school_non_sum_dup
df_closure_tod_wkday_school_non_sum_dup_month['mean_monthly_steps_school_per_month'] = df_closure_tod_wkday_school_non_sum_dup_month.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')

df_closure_tod_wkday_school_non_sum_dup_month_dup = df_closure_tod_wkday_school_non_sum_dup_month.drop_duplicates(subset=['mean_monthly_steps_school_per_month', 'Month_Year_Name'], keep='first')

# Min
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].min()

# Max
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].max()

# Mean
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].mean()

# Std
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].std()

# Users
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
# Statistical test
steps_pre_closure_wkday_school_non_sum = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] == 'Pre Closure']
steps_pre_closure_wkday_school_non_sum['Monthly_Avg'] = steps_pre_closure_wkday_school_non_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')
steps_pre_closure_wkday_school_non_sum['month_num'] = steps_pre_closure_wkday_school_non_sum['Start_Time_Offset'].dt.month
steps_pre_closure_wkday_school_non_sum = steps_pre_closure_wkday_school_non_sum.sort_values(by=['month_num'])
steps_pre_closure_wkday_school_non_sum_col = steps_pre_closure_wkday_school_non_sum[['Month_Year_Name', 'Monthly_Avg']]
steps_pre_closure_wkday_school_non_sum_col_dup = steps_pre_closure_wkday_school_non_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')

steps_dur_closure_wkday_school_non_sum = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] == 'During Closure']
steps_dur_closure_wkday_school_non_sum['Monthly_Avg'] = steps_dur_closure_wkday_school_non_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')
steps_dur_closure_wkday_school_non_sum['month_num'] = steps_dur_closure_wkday_school_non_sum['Start_Time_Offset'].dt.month
steps_dur_closure_wkday_school_non_sum = steps_dur_closure_wkday_school_non_sum.sort_values(by=['month_num'])
steps_dur_closure_wkday_school_non_sum_col = steps_dur_closure_wkday_school_non_sum[['Month_Year_Name', 'Monthly_Avg']]
steps_dur_closure_wkday_school_non_sum_col_dup = steps_dur_closure_wkday_school_non_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_wkday_school_non_sum_col_dup['Monthly_Avg'], steps_pre_closure_wkday_school_non_sum_col_dup['Monthly_Avg'])
print(U1)
print(p*100)

print(p)

In [ ]:
# Summer months weekday steps analysis (pre-, during-, and post-closure)

# Starting dataset used: df_closure_tod_wkday_school
df_closure_tod_wkday_school_only_sum = df_closure_tod_wkday_school[
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jun-2020') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jul-2020') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Aug-2020') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jun-2019') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jul-2019') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Aug-2019') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jun-2021') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jul-2021') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Aug-2021')
]

df_closure_tod_wkday_school_only_sum['Daily_Sum_TOD'] = df_closure_tod_wkday_school_only_sum.groupby(['Start_Time_Date_Offset_x'])['Steps'].transform('sum')
df_closure_tod_wkday_school_only_sum['mean_monthly_steps_school_sum'] = df_closure_tod_wkday_school_only_sum.groupby(['User_UUID'])['Daily_Sum_TOD'].transform('mean')

df_closure_tod_wkday_school_only_sum_dup = df_closure_tod_wkday_school_only_sum.drop_duplicates(subset=['User_UUID', 'mean_monthly_steps_school_sum'], keep='first')

In [ ]:
# Min
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].min()

# Max
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].max()

# Mean
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].mean()

# Std
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].std()

# Users
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
df_closure_tod_wkday_school_only_sum_dup_fig = df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status'] != 'Post Closure']

plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {"Pre Closure": "yellowgreen",
           "During Closure": "lightsalmon"
#            "#2020": "salmon"
          }

ax = sns.boxplot(x="month_only", y="mean_monthly_steps_school_sum", palette=palette, hue='closure_status',
                 data=fig_3_df_closure_tod, linewidth=14, showfliers=False,
                 showmeans=False, width=0.5)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times',
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)


In [ ]:
df_closure_tod_wkday_school_only_sum_dup_fig = df_closure_tod_wkday_school_only_sum_dup[
    df_closure_tod_wkday_school_only_sum_dup['closure_status'] != 'Post Closure']

plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {
    "Pre Closure": "yellowgreen",
    "During Closure": "lightsalmon"
#     "#2020": "salmon"
}

ax = sns.boxplot(x="month_only", y="mean_monthly_steps_school_sum", palette=palette, hue="closure_status",
                 data=fig_3_df_closure_tod, linewidth=14, showfliers=False,
                 showmeans=False, width=0.5)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times',
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)

In [ ]:
# New one value per month
df_closure_tod_wkday_school_only_sum_dup_month = df_closure_tod_wkday_school_only_sum_dup
df_closure_tod_wkday_school_only_sum_dup_month['mean_monthly_steps_school_sum_per_month'] = df_closure_tod_wkday_school_only_sum_dup_month.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')

df_closure_tod_wkday_school_only_sum_dup_month_dup = df_closure_tod_wkday_school_only_sum_dup_month.drop_duplicates(subset=['mean_monthly_steps_school_sum_per_month', 'Month_Year_Name'], keep='first')

# Min
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].min()

# Max
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].max()

# Mean
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].mean()

# Std
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].std()


# Users
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
# Statistical test
steps_pre_closure_wkday_school_only_sum_dup = df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status'] == 'Pre Closure']
steps_pre_closure_wkday_school_only_sum_dup['Monthly_Avg'] = steps_pre_closure_wkday_school_only_sum_dup.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')
steps_pre_closure_wkday_school_only_sum_dup['month_num'] = steps_pre_closure_wkday_school_only_sum_dup['Start_Time_Offset'].dt.month
steps_pre_closure_wkday_school_only_sum_dup = steps_pre_closure_wkday_school_only_sum_dup.sort_values(by=['month_num'])
steps_pre_closure_wkday_school_only_sum_col_dup = steps_pre_closure_wkday_school_only_sum_dup[['Month_Year_Name', 'Monthly_Avg']]
steps_pre_closure_wkday_school_only_sum_col_dup = steps_pre_closure_wkday_school_only_sum_col_dup.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')

steps_dur_closure_wkday_school_only_sum_dup = df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status'] == 'During Closure']
steps_dur_closure_wkday_school_only_sum_dup['Monthly_Avg'] = steps_dur_closure_wkday_school_only_sum_dup.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')
steps_dur_closure_wkday_school_only_sum_dup['month_num'] = steps_dur_closure_wkday_school_only_sum_dup['Start_Time_Offset'].dt.month
steps_dur_closure_wkday_school_only_sum_dup = steps_dur_closure_wkday_school_only_sum_dup.sort_values(by=['month_num'])
steps_dur_closure_wkday_school_only_sum_col_dup = steps_dur_closure_wkday_school_only_sum_dup[['Month_Year_Name', 'Monthly_Avg']]
steps_dur_closure_wkday_school_only_sum_col_dup = steps_dur_closure_wkday_school_only_sum_col_dup.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_dur_closure_wkday_school_only_sum_col_dup

In [ ]:
from scipy.stats import mannwhitneyu

U1, p = mannwhitneyu(steps_dur_closure_wkday_school_only_sum_col_dup['Monthly_Avg'],
                     steps_pre_closure_wkday_school_only_sum_col_dup['Monthly_Avg'])
print(U1)
print(p*100)
print(p)

# Sleep
df_sleep = df[df['Type'] == 'sleepDuration']
len(df_sleep)

df_sleep['User UUID'].nunique()

steps_user_id_list = df_valid_days_daycount['User UUID'].nunique()

df_sleep_step_ids = df_sleep[df_sleep['User UUID'].isin(steps_user_id_list)]
df_sleep_step_ids

df_sleep_step_ids['User UUID'].nunique()

df_valid_days_daycount['User UUID'].nunique()

df_valid_days_daycount['User UUID']

In [ ]:
df_sleep_step_ids['Start Time'] = df_sleep_step_ids['Start Time'].apply(pd.to_datetime)
df_sleep_step_ids['End Time'] = df_sleep_step_ids['End Time'].apply(pd.to_datetime)

df_sleep_step_ids['Start Time Offset'] = df_sleep_step_ids['Start Time'] + pd.to_timedelta(df_sleep_step_ids['Time Zone Offset'], unit='m')
df_sleep_step_ids['End Time Offset'] = df_sleep_step_ids['End Time'] + pd.to_timedelta(df_sleep_step_ids['Time Zone Offset'], unit='m')

df_sleep_step_ids['Start Time Date Offset'] = df_sleep_step_ids['Start Time Offset'].dt.date
df_sleep_step_ids['End Time Date Offset'] = df_sleep_step_ids['End Time Offset'].dt.date
df_sleep_step_ids['month_year'] = df_sleep_step_ids['Start Time Offset'].dt.to_period('M')

df_sleep_step_ids['Mon Year'] = df_sleep_step_ids['Start Time Offset'].dt.strftime('%b-%Y')
df_sleep_step_ids['Month Year Name'] = df_sleep_step_ids['Mon Year']

In [ ]:
df_sleep_copy = df_sleep_step_ids
df_sleep_copy_dup = df_sleep_copy.drop_duplicates(subset=['User UUID', 'Start Time', 'End Time', 'Value'], keep='first')
df_sleep_copy_dup

df_sleep_copy_dup['User UUID'].nunique()

In [ ]:
df_sleep_copy_dup['year_only'] = df_sleep_copy_dup['Start Time Offset'].dt.year
df_sleep_copy_dup['month_only'] = df_sleep_copy_dup['Start Time Offset'].dt.strftime("%B")
df_sleep_copy_dup['Start Time Only Time'] = df_sleep_copy_dup['Start Time Offset'].dt.strftime("%H:%M:%S")

# Convert to datetime format to calculate bedtime date
df_sleep_copy_dup['Start Time Only Time Datetime'] = df_sleep_copy_dup['Start Time Only Time'].apply(pd.to_datetime)

# Get only nighttime values
# diff=445416-410584
# change based on today's date
df_sleep_bedtime_calc_allnight = df_sleep_copy_dup[(df_sleep_copy_dup['Start Time Only Time Datetime'] > dt.datetime(2023, 6, 27, 18, 0, 0)) &
                                                   (df_sleep_copy_dup['Start Time Only Time Datetime'] < dt.datetime(2023, 6, 27, 8, 0, 0))]

df_sleep_bedtime_calc_allnight['Start Time Only Time Datetime'].dt.date.unique()

In [ ]:
# Why twice?
df_sleep_bedtime_calc_allnight['Start Time Only Time Datetime'] = df_sleep_bedtime_calc_allnight['Start Time Only Time'].apply(pd.to_datetime)

from datetime import timedelta

df_sleep_bedtime_calc_allnight_val = df_sleep_bedtime_calc_allnight
# change according to today's date
df_sleep_bedtime_calc_allnight_val.loc[(df_sleep_bedtime_calc_allnight_val['Start Time Only Time Datetime'] 
                                        > dt.datetime(2023, 6, 27, 18, 0, 0)), 'Bedtime_Date'] = 
df_sleep_bedtime_calc_allnight_val['Start Time_Date_Offset']
df_sleep_bedtime_calc_allnight_val.loc[(df_sleep_bedtime_calc_allnight_val['Start Time Only Time Datetime'] 
                                        < dt.datetime(2023, 6, 27, 18, 0, 0)), 'Bedtime_Date'] = 
df_sleep_bedtime_calc_allnight_val['Start_Time_Date_Offset'] - timedelta(days=1)

df_sleep_bedtime_calc_allnight_val['Start Time_Offset_dt'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Start Time_Offset'])
df_sleep_bedtime_calc_allnight_val['End Time_Offset_dt'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['End Time_Offset'])

In [ ]:
df_sleep_bedtime_calc_allnight_val['Start_Time_Offset_dt'] = df_sleep_bedtime_calc_allnight_val['Start_Time_Offset_dt'].astype('int64') // 1e9
df_sleep_bedtime_calc_allnight_val['End_Time_Offset_dt'] = df_sleep_bedtime_calc_allnight_val['End_Time_Offset_dt'].astype('int64') // 1e9

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso'] = df_sleep_bedtime_calc_allnight_val.groupby(['User UUID', 'Bedtime_Date'])['Start_Time_Offset_dt_iso'].transform('min')
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso'] = df_sleep_bedtime_calc_allnight_val.groupby(['User UUID', 'Bedtime_Date'])['End_Time_Offset_dt_iso'].transform('max')

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso'], unit='s')
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso'], unit='s')

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted_dt'] = df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted'].apply(pd.to_datetime)
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted_dt'] = df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted'].apply(pd.to_datetime)

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted_dt'].strftime("%H:%M:%S")
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted_dt'].strftime("%H:%M:%S")

In [7]:
from cmath import rect, phase
from math import radians, degrees

def meanAngle(deg):
    complexDegree = sum(rect(1, radians(d)) for d in deg) / len(deg)
    argument = phase(complexDegree)
    meanAngle = degrees(argument)
    return meanAngle

def meanTime(times):
    t = (time.split(':') for time in times)
    seconds = sum((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    
    day = 24 * 60 * 60
    toAngles = [s * 360. / day for s in seconds]
    meanAsAngle = meanAngle(toAngles)
    meanSeconds = meanAsAngle * day / 360.
    
    if meanSeconds < 0:
        meanSeconds += day
    h, m = divmod(meanSeconds, 3600)
    m, s = divmod(m, 60)
    
    return '%02i:%02i:%02i' % (h, m, s)

In [ ]:
df_sleep_bedtime_calc_allnight_val_copy = df_sleep_bedtime_calc_allnight_val

# get month from bedtime
df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.to_period('M')
df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year_abb'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.strftime('%b')

df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year']

# Do we need this?
df_sleep_bedtime_calc_allnight_val_copy['Mon_Year_Bedtime'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.strftime('%b-%Y')
df_sleep_bedtime_calc_allnight_val_copy['Month_Year_Name_Bedtime'] = df_sleep_bedtime_calc_allnight_val_copy['Mon_Year_Bedtime']

df_sleep_bedtime_calc_allnight_val_copy['Month_Year_Name_Bedtime'] = df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date < dt.date(2020, 3, 15)) &
                                            (df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date >= dt.date(2020, 3, 1)),
                                            'Month_Year_Name_Bedtime'] = '1-14 Mar-2020'
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date < dt.date(2020, 4, 1)) &
                                            (df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date >= dt.date(2020, 3, 15)),
                                            'Month_Year_Name_Bedtime'] = '15-31 Mar-2020'
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date < dt.date(2020, 4, 1)),
                                            'Month_Year_Name_Bedtime'] = df_sleep_bedtime_calc_allnight_val_copy.Month_Year_Name_Bedtime